In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import timeit as ti

In [2]:
class Labeller:
    #Aumi's audio labellers
    
    def __init__(self, name):
        self.name = name
        self.stations = []
        self.time = 0

        print(self.name, "has been added")
        
    def to_dict(self):
        return{
            "Name": self.name,
            "Stations": self.stations,
            "Time": self.time
        }

    def add_station(self, station):
        self.stations.append(Station(station))
        print(self.stations[len(self.stations)-1].name, "added to labeller ", self.name)
    
class Station:
    #Audio monitoring stations
    
    def __init__(self, name):
        self.name = name
        self.events = []
        self.time = 0
    
    def to_dict(self):
        return{
            "Name": self.name,
            "Events": self.events,
            "Time": self.time
        }
    
    def add_event(self, event):
        self.events.append(Event(event))
        print (self.events[len(self.events)-1].name, "added to station ", self.name)
    
class Event:
    #Events registrered in stations
    
    def __init__(self, name):
        self.name = name
        self.time = 0
        self.counter = 0
    
    def to_dict(self):
        return{
            "Name": self.name,
            "Time": self.time,
            "Counter": self.counter
        }
        

In [33]:
#lecture of metadata and acts as a getter of future objects names, as a defaul, this func will metadata1_4.txt
#in the same folder of proyect
def catego_reader():
    #txt reader
    c = pd.read_csv("metadata1_4.txt", sep="\t", engine = "python")
    
    #get unique names from columns
    lab = c.labeller.unique()
    est = c.station.unique()
    evt = c.kind.unique()
    
    #return table and uniques names of columns labeller, station and kind, last one changued from class due conflicts
    return c, lab, est, evt


#modeling de future data container using uniques from catego_reader function 
def model_builder(labellers_names, stations_names, events_names):   
    
    labellers = []
    labellers.clear()
    
    for ln in labellers_names:
        labellers.append(Labeller(ln))
    
    
    for l in labellers:
        for st in stations_names:
            
            l.add_station(st)
    
    for l in labellers:
        for s in l.stations:
            for ev in events_names:
                
                s.add_event(ev)
                
    #function returns a slice of labellers with all instances
    return labellers


#assing table's data to the class model created in model_builder function
#requires table from catego_reader and the slice of Labeller instances from model_build
def data_builder(c, labellers):
    
    for i in range(c.shape[0]):
        for l in labellers:
            
            #compares the l labeller name with the column 5 of table that contains row's labeller name
            if(c.loc[i][5] == l.name):
                
                #if labeller name match, then inspects stationes in l
                for s in l.stations:
                    
                    #compares the station name in labeller instance with column 4 in row that contains station name
                    if(c.loc[i][4] == s.name):
                        
                        #if station names match, then inspects events in station from l instance
                        for e in s.events:
                            
                            #compares the events name in table and station instance                             
                            if(c.loc[i][3] == e.name):
                                
                                #we are in labeller[name], that contains -> Station[name], that contains -> Event[name]
                                
                                #creation/reset var value avoiding data errors
                                last = 0
                                
                                #duration operation
                                last = c.loc[i][2] - c.loc[i][1]
                                
                                #now add the value of duration of event to event, also sum +1 to the counter of event
                                
                                #adding even duration
                                setattr(e, "time", getattr(e, "time") + last)
                                
                                #add +1 to event counter
                                setattr(e, "counter", getattr(e, "counter") + 1)
                                
                                #also adding the time to the station because... why not after all?
                                setattr(s, "time", getattr(s, "time") + last)
                                
                                #adding time of labeling for labeller
                                setattr(l, "time", getattr(l, "time") + last)
                                
                                #setted the value, no need for more looping in this row, break the loops
                                break
                        break
                break
                    
                
    
    return labellers


#creation of data frame of labellers
def labeller_df(labellers):
    
    data = []
    
    for l in labellers:
        sn = []
        
        for s in l.stations:
            sn.append(s.name)
            
        data.append([l.name, l.time, sn.copy()])
    
        
    return pd.DataFrame(data, columns = ["Name", "Labelling Time", "Stations names"])


#creation of data frame at station level
def station_df(labellers):
    
    data = []
    
    for l in labellers:
        for s in l.stations: 
            en = []
            
            for e in s.events:
                en.append(e.name)
            
            
            data.append([l.name, s.name, s.time, en.copy()])
    
    return pd.DataFrame(data, columns = ["Labeller", "Station", "Station Time", "Events names"])
    
            

In [4]:
c, lab, st, evt = catego_reader()

In [5]:
#RUN ONCE!!! OTHERWHISE MODEL WILL BY DUPLICATED, APPLY %reset MANUALLY BEFORE
labellers = model_builder(lab, st, evt)

Hakim_M has been added
Rafael_P has been added
Esteban_G has been added
Cristian_C has been added
Esteban_V has been added
Diego_E has been added
capta_002 added to labeller  Hakim_M
capta_004 added to labeller  Hakim_M
capta_003 added to labeller  Hakim_M
campus_miraflores added to labeller  Hakim_M
control_acustico added to labeller  Hakim_M
capta_002 added to labeller  Rafael_P
capta_004 added to labeller  Rafael_P
capta_003 added to labeller  Rafael_P
campus_miraflores added to labeller  Rafael_P
control_acustico added to labeller  Rafael_P
capta_002 added to labeller  Esteban_G
capta_004 added to labeller  Esteban_G
capta_003 added to labeller  Esteban_G
campus_miraflores added to labeller  Esteban_G
control_acustico added to labeller  Esteban_G
capta_002 added to labeller  Cristian_C
capta_004 added to labeller  Cristian_C
capta_003 added to labeller  Cristian_C
campus_miraflores added to labeller  Cristian_C
control_acustico added to labeller  Cristian_C
capta_002 added to label

alarm added to station  capta_003
truck_idling added to station  capta_003
car_idling added to station  capta_003
bus_moving added to station  capta_003
braking added to station  capta_003
steps added to station  capta_003
bell added to station  capta_003
cutting added to station  capta_003
drilling added to station  capta_003
water added to station  capta_003
airborne added to station  capta_003
bus_idling added to station  capta_003
air_conditioner added to station  capta_003
dog added to station  campus_miraflores
car_moving added to station  campus_miraflores
wind added to station  campus_miraflores
impact added to station  campus_miraflores
crowd added to station  campus_miraflores
talk added to station  campus_miraflores
truck_moving added to station  campus_miraflores
shouting added to station  campus_miraflores
music added to station  campus_miraflores
horn added to station  campus_miraflores
motorcycle_moving added to station  campus_miraflores
motorcycle_idling added to stati

In [6]:
start_time = ti.default_timer()

labellers = data_builder(c, labellers)

print("Function call lasted: ", ti.default_timer()-start_time)

Function call lasted:  13.604715599999999


In [29]:
labeller_df(labellers)

,Name,Labelling Time,Stations
0,Hakim_M,19034.625695,"[capta_002, capta_004, capta_003, campus_miraf..."
1,Rafael_P,12735.138012,"[capta_002, capta_004, capta_003, campus_miraf..."
2,Esteban_G,14921.185669,"[capta_002, capta_004, capta_003, campus_miraf..."
3,Cristian_C,14515.000652,"[capta_002, capta_004, capta_003, campus_miraf..."
4,Esteban_V,15995.104945,"[capta_002, capta_004, capta_003, campus_miraf..."
5,Diego_E,12421.809894,"[capta_002, capta_004, capta_003, campus_miraf..."


In [34]:
station_df(labellers)

,Labeller,Station,Station Time,Events names
0,Hakim_M,capta_002,11783.158841,"[dog, car_moving, wind, impact, crowd, talk, t..."
1,Hakim_M,capta_004,4812.188041,"[dog, car_moving, wind, impact, crowd, talk, t..."
2,Hakim_M,capta_003,1478.683751,"[dog, car_moving, wind, impact, crowd, talk, t..."
3,Hakim_M,campus_miraflores,0.000000,"[dog, car_moving, wind, impact, crowd, talk, t..."
4,Hakim_M,control_acustico,960.595061,"[dog, car_moving, wind, impact, crowd, talk, t..."
5,Rafael_P,capta_002,8867.370927,"[dog, car_moving, wind, impact, crowd, talk, t..."
6,Rafael_P,capta_004,1608.281305,"[dog, car_moving, wind, impact, crowd, talk, t..."
7,Rafael_P,capta_003,1605.413697,"[dog, car_moving, wind, impact, crowd, talk, t..."
8,Rafael_P,campus_miraflores,0.000000,"[dog, car_moving, wind, impact, crowd, talk, t..."
9,Rafael_P,control_acustico,654.072083,"[dog, car_moving, wind, impact, crowd, talk, t..."
